<a href="https://colab.research.google.com/github/rahiakela/deep-reinforcement-learning-with-python/blob/main/3-bellman-equation-and-dynamic-programming/2_solving_frozen_lake_problem_with_policy_iteration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Solving the Frozen Lake Problem with Policy Iteration

Dynamic programming (DP) is a technique for solving complex problems. In DP,
instead of solving a complex problem as a whole, we break the problem into simple sub-problems, then for each sub-problem, we compute and store the solution. If the same subproblem occurs, we don't recompute; instead, we use the already computed solution. Thus, DP helps in drastically minimizing the computation time. It has its applications in a wide variety of fields including computer science, mathematics, bioinformatics, and so on.

Now, we will learn about two important methods that use DP to find the optimal
policy. The two methods are:

- Value iteration
- Policy iteration

Note that dynamic programming is a model-based method meaning that it will help
us to find the optimal policy only when the model dynamics (transition probability) of the environment are known. If we don't have the model dynamics, we cannot apply DP methods.


Reference:

- https://colab.research.google.com/github/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_12_01_ai_gym.ipynb

- https://medium.com/analytics-vidhya/rendering-openai-gym-environments-in-google-colab-9df4e7d6f99f

- https://www.toptal.com/machine-learning/deep-dive-into-reinforcement-learning


## Setup

**Render OpenAI Gym Environments in CoLab**

It is possible to visualize the game your agent is playing, even on CoLab.  This section provides information on how to generate a video in CoLab that shows you an episode of the game your agent is playing. This video process is based on suggestions found [here](https://colab.research.google.com/drive/1flu31ulJlgiRL1dnN2ir8wGh9p7Zij2t).

Begin by installing **pyvirtualdisplay** and **python-opengl**.

In [1]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

Next, we install needed requirements to display an Atari game.

In [2]:
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1

Requirement already up-to-date: setuptools in /usr/local/lib/python3.7/dist-packages (57.0.0)


Next we define functions used to show the video by adding it to the CoLab notebook.

In [3]:
import gym
from gym.wrappers import Monitor
import glob
import io
import base64
import numpy as np

from IPython.display import HTML
from pyvirtualdisplay import Display
from IPython import display as ipythondisplay

display = Display(visible=0, size=(1400, 900))
display.start()

"""
Utility functions to enable video recording of gym environment 
and displaying it.
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

The Gym library allows us to query some of these attributes from environments.  I created the following function to query gym environments.

In [4]:
def query_environment(name):
  env = gym.make(name)
  spec = gym.spec(name)
  print(f"Action Space: {env.action_space}")
  print(f"Observation Space: {env.observation_space}")
  print(f"Max Episode Steps: {spec.max_episode_steps}")
  print(f"Nondeterministic: {spec.nondeterministic}")
  print(f"Reward Range: {env.reward_range}")
  print(f"Reward Threshold: {spec.reward_threshold}")

In [5]:
query_environment("FrozenLake-v0")

Action Space: Discrete(4)
Observation Space: Discrete(16)
Max Episode Steps: 100
Nondeterministic: False
Reward Range: (0, 1)
Reward Threshold: 0.78


## Policy iteration

In the value iteration method, first, we computed the optimal value function by
taking the maximum over the $Q$ function ($Q$ values) iteratively. Once we found the
optimal value function, we used it to extract the optimal policy. 

Whereas in policy iteration we try to compute the optimal value function using the policy iteratively, once we found the optimal value function, we can use it to extract the optimal policy.

First, let's learn how to compute the value function using a policy. Say we have a policy $\pi$ , how can we compute the value function using the policy $\pi$ ?

Here, we can use our Bellman equation. We learned that according to the Bellman equation, we can compute the value function using the policy $\pi$ as the following shows:

$$V^{\pi}(s) = \sum_{a} \pi(a|s) \space \sum_{s^"} P(s^"|s, a)[R(s, a, s^") + \gamma V^{\pi}(s^")]$$

Let's suppose our policy is a deterministic policy, so we can remove the term
$\sum_{a} \pi(a|s)$ from the preceding equation since there is no stochasticity in the policy and rewrite our Bellman equation as:

$$V^{\pi}(s) = \sum_{s^"} P(s^"|s, a)[R(s, a, s^") + \gamma V^{\pi}(s^")]$$

Thus using the above equation we can compute the value function using a policy.
Our goal is to find the optimal value function because once we have found the
optimal value function, we can use it to extract the optimal policy.

We will not be given any policy as an input. So, we will initialize the random policy
and compute the value function using the random policy. Then we check if the
computed value function is optimal or not. It will not be optimal since it is computed based on the random policy.

###Frozen Lake environment

Let's recap the Frozen Lake environment a bit. In the Frozen Lake environment
shown below, the following applies:

* S implies the starting state
* F implies the frozen states
* H implies the hold states
* G implies the goal state

<img src='https://github.com/rahiakela/img-repo/blob/master/deep-reinforcement-learning-with-python/vi-4.png?raw=1'/>

We learned that in the frozen lake environment, our goal is to reach the goal state G from the starting state S without visiting the hole states H. That is, while trying to reach the goal state G from the starting state S if the agent visits the hole state H then it will fall into the hole and die as shown below:

<img src='https://github.com/rahiakela/img-repo/blob/master/deep-reinforcement-learning-with-python/vi-5.png?raw=1'/>

So, the goal of the agent is to reach the state G starting from the state S without visiting the hole states H as shown below:

<img src='https://github.com/rahiakela/img-repo/blob/master/deep-reinforcement-learning-with-python/vi-6.png?raw=1'/>

How can we achieve this goal? That is, how can we reach the state G from S without visiting H? 

We learned that the optimal policy tells the agent to perform correct action in
each state. So, if we find the optimal policy then we can reach the state G from S visiting the state H. 

Okay, how to find the optimal policy? 

We can use the value iteration method we just learned to find the optimal policy.


Remember that all our states (S to G) will be encoded from 0 to 16 and all the four actions - left, down, up, right will be encoded from 0 to 3 in the gym toolkit.

So, we will learn how to find the optimal policy using the value iteration
method so that the agent can reach the state G from S without visiting H.

Now, let's create the frozen lake environment using gym:



In [6]:
env = gym.make('FrozenLake-v0')

Let's look at the frozen lake environment using the render function:

In [7]:
env.render()


SFFF
FHFH
FFFH
HFFG


As we can notice, our agent is in the state S and it has to reach the state G without visiting the states H. So, let's learn how to compute the optimal policy using the value iteration method.

In the value iteration method, we perform two steps:

1. Compute the optimal value function by taking the maximum over the $Q$
function, that is $V^*(s) = max_{\alpha} Q^*(s, a)$
2. Extract the optimal policy from the computed optimal value function

First, let's learn how to compute the optimal value function, and then we will see how to extract the optimal policy from the computed optimal value function.

## Computing the value function using the policy

This step is exactly the same as how we computed the value function in the value
iteration method but with a small difference. Here, we compute the value function using the policy but in the value iteration method, we compute the value function by taking the maximum over Q values. Now, let's learn how to define a function that computes the value function using the given policy.

In [8]:
def compute_value_function(policy):

  # set the number of iterations
  num_iterations = 1000
  # set the threshold number for checking the convergence of the value function
  threshold = 1e-20
  # we also set the discount factor
  gamma = 1.0

  # now, we will initialize the value table, with the value of all states to zero
  value_table = np.zeros(env.observation_space.n)

  # for every iteration
  for i in range(num_iterations):
    # update the value table, that is, we learned that on every iteration, we use the updated value
    # table (state values) from the previous iteration
    updated_value_table = np.copy(value_table)

    '''
    Now, we compute the value function using the given policy. We learned that a value
    function can be computed according to some policy.

    Thus, for each state, we select the action according to the policy and then we update
    the value of the state using the selected action
    '''
    for s in range(env.observation_space.n):
      # Select the action in the state according to the policy
      a = policy[s]
      # Compute the value of the state using the selected action
      value_table[s] = sum([prob*(r + gamma * updated_value_table[s_]) for prob, s_, r, _ in env.P[s][a]])

    '''
    after computing the value table, that is, value of all the states, we check whether the
    difference between value table obtained in the current iteration and previous iteration is
    less than or equal to a threshold value if it is less then we break the loop and return the value table as an accurate value function of the given policy
    '''
    if (np.sum(np.fabs(updated_value_table - value_table)) <= threshold):
      break

  return value_table

Now, that we have computed the optimal value function by taking the maximum over Q values, let's see how to extract the optimal policy from the optimal value function.

## Extracting policy from the value function

This step is exactly the same as how we extracted the policy from the value function in the value iteration method. Thus, similar to what we learned in the value iteration method, we define a function called extract_policy to extract a policy given the value function:

In [9]:
def extract_policy(value_table):
  
  # set the discount factor
  gamma = 1.0

  # First, we initialize the policy with zeros, that is, first, we set the actions for all the states to be zero
  policy = np.zeros(env.observation_space.n)

  '''
  Now, we compute the Q function using the optimal value function obtained from the previous step.
  After computing the Q function, we can extract policy by selecting action which has maximum Q value.
  Since we are computing the Q function using the optimal value function, the policy extracted from the Q function will be the optimal policy.

  As shown below, for each state, we compute the Q values for all the actions in the state and 
  then we extract policy by selecting the action which has maximum Q value.
  '''
  # for each state
  for s in range(env.observation_space.n):
    # compute the Q value of all the actions in the state
    Q_values = [sum([prob*(r + gamma * value_table[s_]) for prob, s_, r, _ in env.P[s][a]]) for a in range(env.action_space.n)]
    # extract policy by selecting the action which has maximum Q value
    policy[s] = np.argmax(np.array(Q_values))

  return policy

That's it! Now, we will see how to extract the optimal policy in our frozen lake environment.

##Putting it all together

First, let's define a function called policy_iteration which takes the environment as a parameter.

In [10]:
def policy_iteration(env):

  # set the number of iterations
  num_iterations = 1000

  # #we learned that in the policy iteration method, we begin by initializing a random policy.
  #so, we will initialize the random policy which selects the action 0 in all the states
  policy = np.zeros(env.observation_space.n) 

  # for every iteration
  for i in range(num_iterations):
    # compute the value function using the policy
    value_function = compute_value_function(policy)
    # Extract the new policy from the computed value function
    new_policy = extract_policy(value_function)

    # If policy and new_policy are the same, then break the loop
    if (np.all(policy == new_policy)):
      break
    
    # else, update the current policy to new_policy
    policy = new_policy

  return policy

Now, let's learn how to perform policy iteration and find the optimal policy in the frozen lake environment.

So, we just feed the frozen lake environment to our policy_iteration function as shown below and get the optimal policy:

In [11]:
optimal_policy = policy_iteration(env)

In [12]:
print(optimal_policy)

[0. 3. 3. 3. 0. 0. 0. 0. 3. 1. 0. 0. 0. 2. 1. 0.]


As we can observe, our optimal policy tells us to perform the correct action in each state. Thus, we learned how to perform the policy iteration method to compute the optimal policy.